In [22]:
import pandas as pd

# Archivo separado por comas
df_mat = pd.read_csv('student-mat.csv', sep=';')
df_por = pd.read_csv('student-por.csv', sep=';')


In [23]:
# Intentamos unificar los datos por student_id, para limpiar el dataframe

df_por_clave = df_por.drop(columns=['G1', 'G2', 'G3', 'absences'])
df_math_clave = df_mat.drop(columns=['G1', 'G2', 'G3', 'absences'])

df_por_clave['clave'] = df_por_clave.apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
df_math_clave['clave'] = df_math_clave.apply(lambda row: '_'.join(row.values.astype(str)), axis=1)

# Crear un student_id para cada estudiante en df_por
df_por['student_id'] = range(1, len(df_por) + 1)

# Crear un diccionario de mapeo de la clave al student_id en df_por
clave_a_id = dict(zip(df_por_clave['clave'], df_por['student_id']))

# Asignar student_id a df_mat usando la clave, dejando NaN donde no hay match
df_mat['student_id'] = df_math_clave['clave'].map(clave_a_id)

# Asignar un id nuevo a los estudiantes sin coincidencia en df_mat
max_id = df_por['student_id'].max()
df_mat['student_id'] = df_mat.apply(
    lambda row: row['student_id'] if pd.notna(row['student_id']) else max_id + row.name + 1, axis=1)


In [24]:
df_mat = df_mat.rename(columns={'absences': 'absences_math', 'G3': 'math_score'})
column_order = ['student_id'] + [col for col in df_mat.columns if col != 'student_id']
df_mat = df_mat[column_order]

df_mat = df_mat.drop(columns=['reason', 'guardian', 'higher', 'goout','health', 'G1','G2'])

duplicados = df_mat.duplicated()
num_duplicados = duplicados.sum()

num_duplicados

0

In [25]:
df_por = df_por.rename(columns={'absences': 'absences_por', 'G3': 'por_score'})
column_order = ['student_id'] + [col for col in df_por.columns if col != 'student_id']
df_por = df_por[column_order]

df_por = df_por.drop(columns=['reason', 'guardian', 'higher', 'goout','health', 'G1','G2'])

duplicados = df_por.duplicated()
num_duplicados = duplicados.sum()

num_duplicados

0

In [26]:
df_por

,student_id,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,...,activities,nursery,internet,romantic,famrel,freetime,Dalc,Walc,absences_por,por_score
0,1,GP,F,18,U,GT3,A,4,4,at_home,...,no,yes,no,no,4,3,1,1,4,11
1,2,GP,F,17,U,GT3,T,1,1,at_home,...,no,no,yes,no,5,3,1,1,2,11
2,3,GP,F,15,U,LE3,T,1,1,at_home,...,no,yes,yes,no,4,3,2,3,6,12
3,4,GP,F,15,U,GT3,T,4,2,health,...,yes,yes,yes,yes,3,2,1,1,0,14
4,5,GP,F,16,U,GT3,T,3,3,other,...,no,yes,no,no,4,3,1,2,0,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,645,MS,F,19,R,GT3,T,2,3,services,...,yes,no,yes,no,5,4,1,2,4,10
645,646,MS,F,18,U,LE3,T,3,1,teacher,...,no,yes,yes,no,4,3,1,1,4,16
646,647,MS,F,18,U,GT3,T,1,1,other,...,yes,yes,no,no,1,1,1,1,6,9
647,648,MS,M,17,U,LE3,T,3,1,services,...,no,no,yes,no,2,4,3,4,6,10


In [27]:
# Concatenar los DataFrames
df_combined = pd.concat([df_por, df_mat]).sort_values(by='student_id')
df_combined.head()

,student_id,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,...,internet,romantic,famrel,freetime,Dalc,Walc,absences_por,por_score,absences_math,math_score
0,1.0,GP,F,18,U,GT3,A,4,4,at_home,...,no,no,4,3,1,1,4.0,11.0,NaN,NaN
0,1.0,GP,F,18,U,GT3,A,4,4,at_home,...,no,no,4,3,1,1,NaN,NaN,6.0,6.0
1,2.0,GP,F,17,U,GT3,T,1,1,at_home,...,yes,no,5,3,1,1,NaN,NaN,4.0,6.0
1,2.0,GP,F,17,U,GT3,T,1,1,at_home,...,yes,no,5,3,1,1,2.0,11.0,NaN,NaN
2,3.0,GP,F,15,U,LE3,T,1,1,at_home,...,yes,no,4,3,2,3,6.0,12.0,NaN,NaN


In [28]:
cols_to_fill = ['absences_por', 'por_score', 'absences_math', 'math_score']

# Reemplazamos valores NaN con el valor máximo dentro del grupo de cada student_id
for col in cols_to_fill:
    df_combined[col] = df_combined.groupby('student_id')[col].transform(lambda x: x.fillna(x.max()))

df_combined = df_combined.drop_duplicates()

df_combined=df_combined.reset_index(drop=True)


In [29]:
df_combined

,student_id,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,...,internet,romantic,famrel,freetime,Dalc,Walc,absences_por,por_score,absences_math,math_score
0,1.0,GP,F,18,U,GT3,A,4,4,at_home,...,no,no,4,3,1,1,4.0,11.0,6.0,6.0
1,2.0,GP,F,17,U,GT3,T,1,1,at_home,...,yes,no,5,3,1,1,2.0,11.0,4.0,6.0
2,3.0,GP,F,15,U,LE3,T,1,1,at_home,...,yes,no,4,3,2,3,6.0,12.0,NaN,NaN
3,4.0,GP,F,15,U,GT3,T,4,2,health,...,yes,yes,3,2,1,1,0.0,14.0,NaN,NaN
4,5.0,GP,F,16,U,GT3,T,3,3,other,...,no,no,4,3,1,2,0.0,13.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
877,1038.0,MS,F,18,U,LE3,T,3,1,teacher,...,yes,no,4,3,1,1,NaN,NaN,0.0,8.0
878,1039.0,MS,F,18,U,GT3,T,1,1,other,...,no,no,1,1,1,1,NaN,NaN,0.0,0.0
879,1040.0,MS,M,20,U,LE3,A,2,2,services,...,no,no,5,5,4,5,NaN,NaN,11.0,9.0
880,1042.0,MS,M,21,R,GT3,T,1,1,other,...,no,no,5,5,3,3,NaN,NaN,3.0,7.0


In [30]:
df_combined.to_csv('df_limpito_EDA.csv')

In [31]:
for column in df_combined.columns:
    # Obtiene los valores únicos y sus frecuencias
    unique_values_counts = df_combined[column].value_counts(dropna=False)
    
    # Imprime el nombre de la columna
    print(f"Columna: {column}")
    # Imprime los valores únicos y sus conteos
    print(unique_values_counts)
    print("\n")

Columna: student_id
student_id
1.0       1
593.0     1
582.0     1
583.0     1
584.0     1
         ..
298.0     1
299.0     1
300.0     1
301.0     1
1044.0    1
Name: count, Length: 882, dtype: int64


Columna: school
school
GP    630
MS    252
Name: count, dtype: int64


Columna: sex
sex
F    511
M    371
Name: count, dtype: int64


Columna: age
age
16    236
17    232
18    195
15    154
19     52
20      9
21      3
22      1
Name: count, dtype: int64


Columna: address
address
U    635
R    247
Name: count, dtype: int64


Columna: famsize
famsize
GT3    618
LE3    264
Name: count, dtype: int64


Columna: Pstatus
Pstatus
T    778
A    104
Name: count, dtype: int64


Columna: Medu
Medu
4    252
2    244
3    198
1    180
0      8
Name: count, dtype: int64


Columna: Fedu
Fedu
2    285
1    222
3    188
4    179
0      8
Name: count, dtype: int64


Columna: Mjob
Mjob
other       334
services    204
at_home     168
teacher     107
health       69
Name: count, dtype: int64


Columna: 

In [32]:
# Continuando con la normalización:

# Cambiar el nombre de la columna 'famsize' a 'famsize_GT3'
df_combined = df_combined.rename(columns={'famsize': 'famsize_GT3'})

# Reemplazar los valores en la columna 'famsize_GT3'
df_combined['famsize_GT3'] = df_combined['famsize_GT3'].replace({'GT3': 'yes', 'LE3': 'no'})

# Familias con más de tres miembros pasan a ser 'yes', y con menos de 3 miembros, 'no'.

## ¿Hacemos lo mismo con Pstatus, scshool, address y sex?

In [ ]:
# hoooolaaaaa